![logo](https://user-images.githubusercontent.com/59526258/124226124-27125b80-db3b-11eb-8ba1-488d88018ebb.png)

> **Copyright (c) 2021 CertifAI Sdn. Bhd.**<br>
 <br>
This program is part of OSRFramework. You can redistribute it and/or modify
<br>it under the terms of the GNU Affero General Public License as published by
<br>the Free Software Foundation, either version 3 of the License, or
<br>(at your option) any later version.
<br>
<br>This program is distributed in the hope that it will be useful,
<br>but WITHOUT ANY WARRANTY; without even the implied warranty of
<br>MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
<br>GNU Affero General Public License for more details.
<br>
<br>You should have received a copy of the GNU Affero General Public License
<br>along with this program.  If not, see <http://www.gnu.org/licenses/>.
<br>

## Table of Content
1. [Filter Methods](#filter)
    1. Univariate Statistics
    2. Correlation Coefficient
    3. Variance Threshold 
    <br/><br/>
2. [Wrapper Methods](#wrapper)
    1. Recursive Feature Elimination (RFE)
    2. Forward Feature Selection
    3. Backward Feature Elimination
    4. Exhaustive Feature Selection
    <br/><br/>
3. [Embedded Methods](#embedded)
    1. LASSO (Least Absolute Shrinkage and Selection Operator)
        - `LinearSVC` with `SelectFromModel`
        - `Lasso` with Thresholding
    2. Tree-based Model
        - `ExtraTreesClassifier` with `SelectFromModel`
        - `ExtraTreesClassifier` with Thresholding

## Learning Outcome
After going through this notebook, you should be able to: 
1. Perform filter methods to select features
2. Perform wrapper methods to select features
3. Perform embedded methods to select features

# Feature Selection
Feature selection is a **process where you select a number of features in your data that contribute most to the prediction** or remove the irrelevant and insignificant features. This helps to improve generalization, reduce overfitting, and even improve accuracy of model in some cases. It also saves the computational resources needed as you can train with smaller set of features.

There are three types of feature selection: **Filter methods** (univariate statistics, Pearson correlation, variance thresholding), **Wrapper methods** (forward, backward, and exhaustive selection), and **Embedded methods** (Lasso, Ridge, Decision Tree). 

We will go into an explanation of each with examples below.

In [ ]:
# load data
import pandas as pd
from sklearn.datasets import load_wine

# models
from sklearn.svm import LinearSVC
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier

# filter
from sklearn.feature_selection import SelectKBest, chi2, VarianceThreshold, SelectFromModel

# wrapper
from sklearn.feature_selection import RFE, SequentialFeatureSelector
from mlxtend.feature_selection import ExhaustiveFeatureSelector

# embedded
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
# Load data
wine_data = load_wine()
df = pd.DataFrame(data=wine_data.data,
                  columns=wine_data.feature_names)

# Adding the target variable
df["target"] = wine_data.target
df.head()

In [ ]:
# Input and output features
X = df.drop("target", axis=1)
y = df["target"]

In [ ]:
# Check data shape
X.shape

We have 178 samples and 13 variables in the dataset.

## Filter Methods <a id="filter"></a>
Features are filtered based on general characteristics (some metric such as correlation) of the dataset. Filter method is performed without any predictive model.

It is faster and usually the better approach when the number of features are huge. Avoids overfitting but sometimes may fail to select best features.

### Univariate Statistics
Univariate feature selection works by selecting the best features based on univariate statistical tests. 

Scikit-learn univariate feature selection:

- `SelectKBest` removes all but the  highest scoring features

- `SelectPercentile` removes all but a user-specified highest scoring percentage of features 

- `GenericUnivariateSelect` allows to perform univariate feature selection with a configurable strategy

These objects take as input a scoring function that returns univariate scores and p-values (or only scores for `SelectKBest` and `SelectPercentile`):

- For regression: `f_regression`, `mutual_info_regression`
- For classification: `chi2`, `f_classif`, `mutual_info_classif`

We will show an example of `SelectKBest` below.

In [ ]:
#TODO : Use SlectKBest to select 4 features
kbest = 
kbest.

# Feature scores
print("Feature score: ", kbest.scores_)

new_features = kbest.transform(X)

new_features[:5, :]

### Correlation Coefficient
Correlation is a measure of the linear relationship of 2 or more variables. We would assume that the **good variables** are **highly correlated** with the target. Also, sometimes we would want to remove either one of the two variables that are highly correlated. 
<br><br>
<div align="center">
  <img alt="Several sets of (x, y) points, with the correlation coefficient of x and y for each set." src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d4/Correlation_examples2.svg/1920px-Correlation_examples2.svg.png" width="400" height="200"><br>
  <sup>Sample datasets and their pearson correlation coefficients.<sup>
</div>
      
We will show an example that drop the variable which has a lower *Pearson* correlation coefficient value with the target variable. We need to set an absolute value, for example, 0.4 as the threshold for selecting the variables.
      
You may use other correlation coefficient such as *Spearman* or *Kendall*.

In [ ]:
# Pearson correlation coefficient
corr = df.corr()["target"].sort_values(ascending=False)[1:]

# Absolute for positive values
abs_corr = abs(corr)

#TODO : Threshold for features to keep
new_features = 
new_features

# new_df = df[new_features.index]
# new_df.head()

### Variance Threshold
`VarianceThreshold` is a simple baseline approach to feature selection. It removes all features whose variance doesn’t meet some threshold. By default, it removes all zero-variance features, i.e. features that have the same value in all samples.

The estimator only works with numeric data and it will raise an error if there are categorical features present in the dataframe.

In [ ]:
#TODO : VarianceThreshold to keep features with variance higher than 0.7 
vt = 
new_features = vt.

print(new_features.shape)
new_features[:5, :]

## Wrapper Methods <a id="wrapper"></a>
The feature selection algorithm is like a wrapper around the predictive model algorithm and uses the same model to select best features. 

It tends to give better performance as it search through a wider variety of predictor subsets. Though computationally expensive and prone to overfitting.

### Recursive Feature Elimination (RFE)
Recursive feature elimination (RFE) selects features by recursively considering smaller and smaller sets of features. 

First, the estimator is trained on the initial set of features and the importance of each feature is obtained. Then, the least important features are pruned from current set of features. That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached.

In [ ]:
# Defining model to build
lin_reg = LinearRegression()

#TODO : Create a RFE and select 6 features
rfe = 
rfe.

# Summarize the selection of the attributes
print("Num Features: %s" % (rfe.n_features_))
print("Selected Features: %s" % (rfe.support_))
print("Feature Ranking: %s" % (rfe.ranking_))

new_features = rfe.transform(X)
new_features[:5, :]

### Forward Feature Selection
The procedure starts with an empty set of features. The best of the original features is determined and added to the reduced set. 

In [ ]:
#TODO : Create a forward feature selector and select 4 features
ffs = 
ffs.

new_features = ffs.transform(X)
new_features[:5, :]

### Backward Feature Elimination
 The procedure starts with the full set of attributes. At each step, it removes the worst attribute remaining in the set.

In [ ]:
#TODO : Create a backward feature selector and select 4 features
bfs = 
bfs.

new_features = bfs.transform(X)
new_features[:5, :]

### Exhaustive Feature Selection
This is a brute-force evaluation of each feature subset. It tries every possible combination of the variables and returns the best performing subset but also takes longer time.

In [ ]:
%%time
knn = KNeighborsClassifier(n_neighbors=3)

#TODO : Create an exhaustive feature selector and select 4 features
efs = 
efs.

new_features = efs.transform(X)
new_features[:5, :]

## Embedded Methods <a id="embedded"></a>
Feature selection process is embedded in the learning or the model building phase. 

It is less computationally expensive than wrapper method and less prone to overfitting. However, it is model specific (only certain models have intrinsic feature selection).

**SelectFromModel**

*Scikit-learn* provides `SelectFromModel` which is a meta-transformer that can be used alongside any estimator that assigns importance to each feature through a specific attribute (such as `coef_`, `feature_importances_`).

**L1-based models** and **tree-based models** can be used along with `SelectFromModel` to select the non-zero coefficients and discard irrelevant features. Otherwise, we can select features with simple thresholding.

In the example below, we will show two feature selection methods for both L1-based models and tree-based models:
1. Model with `SelectFromModel`
2. Model with Thresholding

### LASSO (Least Absolute Shrinkage and Selection Operator)
This type of regularization (L1) can lead to zero coefficients. Lasso selects the only some feature while reduces the coefficients of others to zero. 
<div align="center">
  <img alt="" src="https://user-images.githubusercontent.com/79887667/134531133-7bd90082-ace5-47f9-a4b4-20ab6fca1505.png" width="400" height="200"><br>
  <sup>Cost function for Lasso regression<sup>
</div>
Linear models penalized with the L1 norm will have sparse solutions: many of their estimated coefficients are zero. Thus, they are feature selectors by nature. 

We can use a linear model with `penalty="l1"` or the `Lasso` module from *scikit-learn*, which both are linear model trained with L1 prior as regularizer. 

1. `LinearSVC` with `SelectFromModel`

In [ ]:
#TODO : Define a LinearSVC model with "l1" penalty
lsvc = 

#TODO : Select features from L1-based model with SelectFromModel
model = 
new_features = model.

print(new_features.shape)
new_features[:5, :]

2. `Lasso` with Thresholding

In [ ]:
#TODO : Train a Lasso model which includes feature selection internally
lasso = 
lasso.

# Perform feature selection
new_features = [feature for feature, weight in zip(
    X.columns.values, lasso.coef_) if weight != 0]

print(len(new_features))
new_features

In order to better understand the effect of regularization, here is a helper function that will  print out the function fit by the regression model.

In [ ]:
# A helper method for pretty-printing the coefficients
def pretty_print_coefs(coefs, names=None, sort=False):
    if names == None:
        names = ["X%s" % x for x in range(len(coefs))]
    lst = zip(coefs, names)
    if sort:
        lst = sorted(lst,  key=lambda x: -np.abs(x[0]))
    return " + ".join("%s * %s" % (round(coef, 3), name)
                      for coef, name in lst)

In [ ]:
print("Lasso model:", pretty_print_coefs(lasso.coef_))

### Tree-based Model
Tree-based estimators (see the `sklearn.tree` module and forest of trees in the `sklearn.ensemble` module) can be used to compute impurity-based feature importances.

Feature importances are provided by the fitted attribute `feature_importances_`. Feature importance is calculated as the decrease in node impurity weighted by the probability of reaching that node. The node probability can be calculated by the number of samples that reach the node, divided by the total number of samples. The higher the value the more important the feature.

When training a tree, the more a feature decreases the impurity, the more important the feature is. Generally, features that are selected at the top of the trees are more important than features that are selected at the end nodes of the trees, as the top splits lead to bigger information gains.

1. `ExtraTreesClassifier` with `SelectFromModel`

In [ ]:
#TODO : Define a ExtraTreesClassifier model
clf = ExtraTreesClassifier(n_estimators=50, random_state=1).fit(X, y)

#TODO : Select features from tree-based model with SelectFromModel
model = 
new_features = model.

print("Feature importance: ", clf.feature_importances_, end="\n\n")
print(new_features.shape)
new_features[:5, :]

2. `ExtraTreesClassifier` with Thresholding

In [ ]:
#TODO : Train an ExtraTreesClassifier which calculate the feature importances internally
model = ExtraTreesClassifier(n_estimators=10, random_state=1)
model.fit(X, y)

print(model.feature_importances_)

# Perform feature selection
new_features = [feature for feature, weight in zip(
    X.columns.values, model.feature_importances_) if weight > 0.1]

print(len(new_features))
new_features